In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

In [3]:
path = os.path.realpath('/usr/share/div_dataset/SW-106.csv')
dataset = pd.read_csv(path)

In [5]:
dataset

,when,hwid,metric,value
0,1601510422405,SW-106,S49,0
1,1601510422405,SW-106,S50,0
2,1601510422405,SW-106,S53,0
3,1601510422405,SW-106,S54,0
4,1601510422405,SW-106,S55,1
...,...,...,...,...
26187744,1616830116654,SW-106,S143,0
26187745,1616830116654,SW-106,S140,73
26187746,1616830116654,SW-106,S138,51
26187747,1616830116654,SW-106,S137,82


In [6]:
# Pivot wider: spread 'metric' into columns, values from 'value'
dataset = dataset.pivot(index='when', columns='metric', values='value').reset_index()

# Convert 'when' (in milliseconds) to datetime
dataset['datetime'] = pd.to_datetime(dataset['when'], unit='ms', utc=True)

# Reorder columns: put 'datetime' first
cols = ['datetime'] + [col for col in dataset.columns if col not in ['datetime']]
dataset = dataset[cols]

# Sort by 'when'
dataset = dataset.sort_values('when').reset_index(drop=True)

In [7]:
dataset

metric,datetime,when,A5,A9,ComError,P1,P10,P15,P16,P17,...,SA12,SA2,SA3,SA4,SA5,SA6,SA7,SA8,SA9,SW
0,2020-10-01 00:00:22.405000+00:00,1601510422405,NaN,NaN,NaN,2.0,450.0,0.0,100.0,400.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2020-10-01 00:00:25.346000+00:00,1601510425346,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,2020-10-01 00:00:52.526000+00:00,1601510452526,NaN,NaN,NaN,2.0,450.0,0.0,100.0,400.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,2020-10-01 00:00:55.485000+00:00,1601510455485,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4,2020-10-01 00:01:22.806000+00:00,1601510482806,NaN,NaN,NaN,2.0,450.0,0.0,100.0,400.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
398034,2021-03-27 07:27:35.592000+00:00,1616830055592,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
398035,2021-03-27 07:28:02.953000+00:00,1616830082953,NaN,NaN,NaN,0.0,450.0,0.0,140.0,450.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
398036,2021-03-27 07:28:05.771000+00:00,1616830085771,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
398037,2021-03-27 07:28:33.937000+00:00,1616830113937,NaN,NaN,NaN,0.0,450.0,0.0,140.0,450.0,...,0.0,16.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [8]:
# Fill NA with nearest non-NA values in both directions
cols_to_fill = [col for col in dataset.columns if col.startswith(('S', 'P', 'A', 'ComError'))]
dataset[cols_to_fill] = dataset[cols_to_fill].ffill().bfill()

# Compute time differences (in seconds)
dataset['time_diff'] = dataset['datetime'].diff().dt.total_seconds()

# Remove the first row with NA time_diff
time_differences = dataset['time_diff'].dropna()

# Move time_diff to the front
cols = ['time_diff'] + [col for col in dataset.columns if col != 'time_diff']
dataset = dataset[cols]

# Print summary statistics
print("Statistiche delle differenze temporali (in secondi):")
print(time_differences.describe())

# Create frequency table
freq_df = time_differences.value_counts().reset_index()
freq_df.columns = ['differenza_secondi', 'frequenza']
freq_df = freq_df.sort_values('differenza_secondi').reset_index(drop=True)

dataset = dataset.copy()

/tmp/ipykernel_149834/3736259583.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dataset['time_diff'] = dataset['datetime'].diff().dt.total_seconds()


Statistiche delle differenze temporali (in secondi):
count    3.980380e+05
mean     3.848802e+01
std      8.665382e+03
min      2.040000e+00
25%      2.880000e+00
50%      2.352050e+01
75%      2.743700e+01
max      5.003774e+06
Name: time_diff, dtype: float64


In [9]:
dataset

metric,time_diff,datetime,when,A5,A9,ComError,P1,P10,P15,P16,...,SA12,SA2,SA3,SA4,SA5,SA6,SA7,SA8,SA9,SW
0,NaN,2020-10-01 00:00:22.405000+00:00,1601510422405,0.0,0.0,1.0,2.0,450.0,0.0,100.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2.941,2020-10-01 00:00:25.346000+00:00,1601510425346,0.0,0.0,1.0,2.0,450.0,0.0,100.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,27.180,2020-10-01 00:00:52.526000+00:00,1601510452526,0.0,0.0,1.0,2.0,450.0,0.0,100.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,2.959,2020-10-01 00:00:55.485000+00:00,1601510455485,0.0,0.0,1.0,2.0,450.0,0.0,100.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,27.321,2020-10-01 00:01:22.806000+00:00,1601510482806,0.0,0.0,1.0,2.0,450.0,0.0,100.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
398034,2.845,2021-03-27 07:27:35.592000+00:00,1616830055592,0.0,0.0,0.0,0.0,450.0,0.0,140.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
398035,27.361,2021-03-27 07:28:02.953000+00:00,1616830082953,0.0,0.0,0.0,0.0,450.0,0.0,140.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
398036,2.818,2021-03-27 07:28:05.771000+00:00,1616830085771,0.0,0.0,0.0,0.0,450.0,0.0,140.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
398037,28.166,2021-03-27 07:28:33.937000+00:00,1616830113937,0.0,0.0,0.0,0.0,450.0,0.0,140.0,...,0.0,16.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [10]:
dataset[dataset['time_diff'] > 120]

metric,time_diff,datetime,when,A5,A9,ComError,P1,P10,P15,P16,...,SA12,SA2,SA3,SA4,SA5,SA6,SA7,SA8,SA9,SW
50,145.599,2020-10-01 00:14:53.888000+00:00,1601511293888,0.0,0.0,1.0,2.0,450.0,0.0,100.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
59,129.879,2020-10-01 00:19:05.243000+00:00,1601511545243,0.0,0.0,1.0,2.0,450.0,0.0,100.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
78,126.885,2020-10-01 00:25:45.369000+00:00,1601511945369,0.0,0.0,1.0,2.0,450.0,0.0,100.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
81,124.388,2020-10-01 00:28:20.298000+00:00,1601512100298,0.0,0.0,1.0,2.0,450.0,0.0,100.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
94,125.569,2020-10-01 00:33:28.165000+00:00,1601512408165,0.0,0.0,1.0,2.0,450.0,0.0,100.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397853,144.700,2021-03-27 06:34:05.110000+00:00,1616826845110,0.0,0.0,0.0,0.0,450.0,0.0,140.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
397854,127.312,2021-03-27 06:36:12.422000+00:00,1616826972422,0.0,0.0,0.0,0.0,450.0,0.0,140.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
397947,123.977,2021-03-27 07:01:45.815000+00:00,1616828505815,0.0,0.0,0.0,0.0,450.0,0.0,140.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
397961,148.776,2021-03-27 07:07:19.647000+00:00,1616828839647,0.0,0.0,0.0,0.0,450.0,0.0,140.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
